# Société 

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import random

Dans la première matrice on place les non-consommateurs de drogues (modélisé avec le chiffre 1) et les consommateurs (modélisé avec le chiffre 0). 
On a d'abord une matrice composée uniquement de non-consommateurs, puis la répartition des consommateurs se fait grace a une certaine probabilité qu'on applique 

In [7]:
def tableauBase(P):
    """les 0 sont les non-cosommateurs et 1 les consommateurs."""
    tableau_de_zero_un = np.zeros((50,50), dtype='i')
    for i in range(len(tableau_de_zero_un)):
        for j in range(len(tableau_de_zero_un)):
            if random.random() < P :
                tableau_de_zero_un[i,j] = 1
    #print("\n\n\t\t VILLE BASIQUE\n\n\n")
    print(tableau_de_zero_un)
    
    return tableau_de_zero_un
    
tab_base = tableauBase(0.5)

print("nb pers :",verifNb(tab_base))

[[0 0 1 ... 0 1 0]
 [0 0 0 ... 0 0 1]
 [1 1 0 ... 1 0 0]
 ...
 [0 0 1 ... 1 0 0]
 [1 0 1 ... 0 0 0]
 [1 1 1 ... 0 1 0]]


NameError: name 'verifNb' is not defined

In [6]:
def voisins(tab, rowNumber, columnNumber):
    '''
    Petite fonction pour récupérer les voisins
    '''
    N = []
    for i in range(rowNumber-1, rowNumber+2):
        for j in range(columnNumber-1, columnNumber+2):
            if  (i >= 0 and i < tab.shape[0]) and (j >= 0 and j < tab.shape[1]) and ((i,j)!=(rowNumber,columnNumber)):
                N.append(tab[i,j])
    return N


AttributeError: 'int' object has no attribute 'shape'